In [1]:
using Plots
using ParticleFilters
using Distributions
using StaticArrays
using LinearAlgebra
using Random
using StatsBase
#using Reel
using SparseArrays
using GridInterpolations
using DataStructures
using DataFrames
using CSV
using Distributed

In [17]:

include("atan2.jl")
include("obs_rel.jl")
include("polargrid_rel_qual2.jl")
rng = MersenneTwister(2);
#

In [3]:
# random walk

POS_0 = [6.0, 60.0]
CRS_0 = 90 # target's course
SPD_0 = 1 # 1 or 2
TGT_SPD = 1

#input is course in degrees and rng
#returns next course in degrees
function next_crs(crs,rng)
    if rand(rng) < .9
        return crs
    end
    crs = (crs + rand(rng,[-1,1])*30) % 360
    if crs < 0 crs += 360 end
    return crs
end

# state as tuple (x, y, crs, spd) of target (spd of o/s)
function f(state, control, rng)
    r, θ, crs, spd = state
    θ += control[1]
    spd = control[2]
    if θ < 0 θ += 360 end
    θ = θ % 360
    crs -= control[1]
    if crs < 0 crs += 360 end
    crs = crs % 360
    x = r*cos(π/180*θ)
    y = r*sin(π/180*θ)
    pos = [x + TGT_SPD*cos(π/180*crs) - spd, y + 
        TGT_SPD*sin(π/180*crs)]
    crs = next_crs(crs,rng)
    r = sqrt(pos[1]^2 + pos[2]^2)
    θ = atan2(pos[1],pos[2])*180/π
    if θ < 0 θ += 360 end
    return (r, θ, crs, spd)::NTuple{4, Real}
end
#

f (generic function with 1 method)

In [44]:
function r(s)
    range = s[1]
    if range > 150 return -.2 end  # reward to not lose track of contact
    if range <= 10 return -100 end  # collision avoidance
    return .1  # being in "sweet spot" maximizes reward
end
#

r (generic function with 1 method)

In [5]:
#angles = [0, 30, 60, 90, 120, 150, 210, 240, 270, 300, 330]

statespace = thestates
actionspace = ((30,1), (0,1), (-30,1), (30, 2), (0,2), (-30,2))

action_index(a) = trunc(Int, 2*(a[1]/30+1) + a[2])
actions_ = ((-30,1), (-30, 2), (0, 1), (0, 2), (30, 1), (30, 2))



((-30, 1), (-30, 2), (0, 1), (0, 2), (30, 1), (30, 2))

In [6]:
# returns vector rather than Tuple, for particle filter
function f2(x, u, rng)
    temp = [i for i in f(x, u, rng)]
    return temp
end

f2 (generic function with 1 method)

In [49]:
totals = [0.0]
θ = zeros(length(grid),6);
e = sparse(zeros(length(grid),6))
#θ = [r(Tuple(ind2x(grid, j))) for j in 1:length(grid), i in 1:6];

In [48]:

N = 1000
model = ParticleFilterModel{Vector{Float64}}(f2, g)
pfilter = SIRParticleFilter(model, N);
α = 0.3
γ = 0.98
ϵ = 0.3
x = [20, 60, 90, 1, 1];
λ = 0.98
b = ParticleCollection([[20, 60, 90, 1] for i in 1:N]);
counter = 0


0

In [ ]:

## Q-learning loop
#plots = []
#betas = Deque{Array}()
#β = zeros(length(grid),6);
# this version uses Greg's weighted grid 3 -- averages before interpolating
# local approximation instead of global approximation
totals = [0.0]
epochs = 100000
epochsize = 200

total = 0

k = 15
last = 0
x = [rand(rng, 25:150), rand(rng,0:359), rand(rng,0:11)*30, 1, 1];     
b = ParticleCollection([x[1:4] for i in 1:N]);     
ξ = sparse(weighted_grid_3(b))
uu = next_action([transpose(θ[:,j])*ξ for j in 1:size(θ)[2]], ϵ, rng)
u = uu[1]
rando = 0
rando += uu[2]

for i in 1:(epochsize*epochs)
    counter += 1
    
    e[:,u] += ξ
    last = transpose(θ[:,u])*ξ
    
    # choose next action
    #uu = softmax_action([transpose(θ[:,j])*ξ for j in 1:size(θ)[2]], k, rng)
    uu = next_action([transpose(θ[:,j])*ξ for j in 1:size(θ)[2]], ϵ, rng)
    u = uu[1]
    rando += uu[2]
    a_star = uu[3]
    
    #observe new state and reward
    xp = f2(x, actions_[u], rng)
    y = h(xp, rng)
    b = update(pfilter, b, actions_[u], y)
    rew = r(Tuple(xp))
       
    ξ = sparse(weighted_grid_3(b))
     
    total += rew
        
    #cur = (rew + γ * max2([transpose(θ[:,j])*ξ for j in 1:size(θ)[2]], rng))
    
    #argmax state-action pair, not realized state-action pair?
    cur = transpose(θ[:,a_star])*ξ
    
    δ = rew + γ * cur - last
    
    θ += α * δ * e
    #if uu[2] == 0
    e *= γ * λ
    #else
    #    e = sparse(zeros(length(grid),6))
    #end
    
        
    if (counter % epochsize == 0) || (xp[1] <= 10)
        push!(totals, total*500/epochsize)
        println("--------- CURRENT: ", round(total*500/epochsize, sigdigits=4), " AVG: ", round(mean(totals), sigdigits=4), " Epoch: ", 
            trunc(Int, counter/epochsize), " -----------")
        total = 0
        rando = 0
        xp = [rand(rng, 25:150), rand(rng,0:359), rand(rng,0:11)*30, rand(rng, [1,2]), 1];
        b = ParticleCollection([xp[1:4] for i in 1:N]);
        ξ = sparse(weighted_grid_3(b))
        uu = next_action([transpose(θ[:,j])*ξ for j in 1:size(θ)[2]], ϵ, rng)
        u = uu[1]
        running_avg = mean(totals)
        if running_avg > 0 
            k = max(running_avg, 12)
        else
            k = 12
        end
    end
    if counter % (epochsize*5) == 0
        thetaframe = DataFrame(θ)
        CSV.write("current475.csv", thetaframe)
    end
    
    x = xp
    
    
    #plotting
    #r_ = [row[1] for row in particles(b)]
    #theta = [row[2] for row in particles(b)]*π/180
    #x_theta = x[2]*π/180
    #x_r = x[1 ]
    
    #print(".")
    #plt = plot(proj=:polar, lims=(0,200), size=(1000,1000))
    #scatter!(theta, r_, markersize=1, label="particles")
    #scatter!([x_theta], [x_r], markersize=3, label="target")
          
    #push!(plots, plt)
    
end


--------- CURRENT: 50.0 AVG: 25.0 Epoch: 1 -----------
--------- CURRENT: 19.25 AVG: 23.08 Epoch: 2 -----------
--------- CURRENT: 50.0 AVG: 29.81 Epoch: 3 -----------
--------- CURRENT: 50.0 AVG: 33.85 Epoch: 4 -----------
--------- CURRENT: 50.0 AVG: 36.54 Epoch: 5 -----------
--------- CURRENT: 50.0 AVG: 38.46 Epoch: 6 -----------
--------- CURRENT: 50.0 AVG: 39.91 Epoch: 7 -----------
--------- CURRENT: 50.0 AVG: 41.03 Epoch: 8 -----------
--------- CURRENT: 50.0 AVG: 41.93 Epoch: 9 -----------
--------- CURRENT: 50.0 AVG: 42.66 Epoch: 10 -----------
--------- CURRENT: 50.0 AVG: 43.27 Epoch: 11 -----------
--------- CURRENT: 50.0 AVG: 43.79 Epoch: 12 -----------
--------- CURRENT: -214.0 AVG: 25.38 Epoch: 12 -----------
--------- CURRENT: 1.75 AVG: 23.8 Epoch: 13 -----------
--------- CURRENT: 49.25 AVG: 25.39 Epoch: 14 -----------
--------- CURRENT: 50.0 AVG: 26.84 Epoch: 15 -----------
--------- CURRENT: 50.0 AVG: 28.13 Epoch: 16 -----------
--------- CURRENT: 50.0 AVG: 29.28 Epo

--------- CURRENT: -90.25 AVG: 3.602 Epoch: 132 -----------
--------- CURRENT: 50.0 AVG: 3.922 Epoch: 133 -----------
--------- CURRENT: 50.0 AVG: 4.238 Epoch: 134 -----------
--------- CURRENT: -4.0 AVG: 4.182 Epoch: 135 -----------
--------- CURRENT: 50.0 AVG: 4.492 Epoch: 136 -----------
--------- CURRENT: 50.0 AVG: 4.797 Epoch: 137 -----------
--------- CURRENT: 50.0 AVG: 5.098 Epoch: 138 -----------
--------- CURRENT: 50.0 AVG: 5.396 Epoch: 139 -----------
--------- CURRENT: 50.0 AVG: 5.689 Epoch: 140 -----------
--------- CURRENT: 47.0 AVG: 5.959 Epoch: 141 -----------
--------- CURRENT: 32.75 AVG: 6.133 Epoch: 142 -----------
--------- CURRENT: 48.5 AVG: 6.406 Epoch: 143 -----------
--------- CURRENT: 50.0 AVG: 6.686 Epoch: 144 -----------
--------- CURRENT: -74.5 AVG: 6.169 Epoch: 145 -----------
--------- CURRENT: -9.25 AVG: 6.071 Epoch: 146 -----------
--------- CURRENT: 50.0 AVG: 6.347 Epoch: 147 -----------
--------- CURRENT: 50.0 AVG: 6.62 Epoch: 148 -----------
--------- 

--------- CURRENT: 50.0 AVG: 6.235 Epoch: 390 -----------
--------- CURRENT: 50.0 AVG: 6.338 Epoch: 391 -----------
--------- CURRENT: 50.0 AVG: 6.44 Epoch: 392 -----------
--------- CURRENT: 50.0 AVG: 6.542 Epoch: 393 -----------
--------- CURRENT: 39.5 AVG: 6.619 Epoch: 394 -----------
--------- CURRENT: 0.5 AVG: 6.605 Epoch: 395 -----------
--------- CURRENT: 50.0 AVG: 6.705 Epoch: 396 -----------
--------- CURRENT: 50.0 AVG: 6.806 Epoch: 397 -----------
--------- CURRENT: 50.0 AVG: 6.905 Epoch: 398 -----------
--------- CURRENT: 50.0 AVG: 7.005 Epoch: 399 -----------
--------- CURRENT: 50.0 AVG: 7.103 Epoch: 400 -----------
--------- CURRENT: 50.0 AVG: 7.202 Epoch: 401 -----------
--------- CURRENT: 50.0 AVG: 7.3 Epoch: 402 -----------
--------- CURRENT: 50.0 AVG: 7.397 Epoch: 403 -----------
--------- CURRENT: 50.0 AVG: 7.494 Epoch: 404 -----------
--------- CURRENT: -79.0 AVG: 7.298 Epoch: 405 -----------
--------- CURRENT: 50.0 AVG: 7.395 Epoch: 406 -----------
--------- CURRENT

--------- CURRENT: 50.0 AVG: 6.791 Epoch: 523 -----------
--------- CURRENT: 50.0 AVG: 6.867 Epoch: 524 -----------
--------- CURRENT: -54.25 AVG: 6.759 Epoch: 525 -----------
--------- CURRENT: 50.0 AVG: 6.835 Epoch: 526 -----------
--------- CURRENT: 50.0 AVG: 6.911 Epoch: 527 -----------
--------- CURRENT: -36.25 AVG: 6.835 Epoch: 528 -----------
--------- CURRENT: -91.0 AVG: 6.664 Epoch: 529 -----------
--------- CURRENT: 50.0 AVG: 6.74 Epoch: 530 -----------
--------- CURRENT: 50.0 AVG: 6.815 Epoch: 531 -----------
--------- CURRENT: -79.0 AVG: 6.666 Epoch: 532 -----------
--------- CURRENT: -21.25 AVG: 6.618 Epoch: 533 -----------
--------- CURRENT: 50.0 AVG: 6.693 Epoch: 534 -----------
--------- CURRENT: 50.0 AVG: 6.768 Epoch: 535 -----------
--------- CURRENT: 50.0 AVG: 6.842 Epoch: 536 -----------
--------- CURRENT: 50.0 AVG: 6.917 Epoch: 537 -----------
--------- CURRENT: 50.0 AVG: 6.991 Epoch: 538 -----------
--------- CURRENT: 50.0 AVG: 7.065 Epoch: 539 -----------
-------

--------- CURRENT: 18.5 AVG: 6.347 Epoch: 654 -----------
--------- CURRENT: 36.5 AVG: 6.39 Epoch: 655 -----------
--------- CURRENT: 50.0 AVG: 6.451 Epoch: 656 -----------
--------- CURRENT: 47.0 AVG: 6.508 Epoch: 657 -----------
--------- CURRENT: 50.0 AVG: 6.57 Epoch: 658 -----------
--------- CURRENT: 50.0 AVG: 6.631 Epoch: 659 -----------
--------- CURRENT: 24.5 AVG: 6.656 Epoch: 660 -----------
--------- CURRENT: 50.0 AVG: 6.716 Epoch: 661 -----------
--------- CURRENT: -219.0 AVG: 6.401 Epoch: 661 -----------
--------- CURRENT: 18.75 AVG: 6.418 Epoch: 662 -----------
--------- CURRENT: 28.25 AVG: 6.448 Epoch: 663 -----------
--------- CURRENT: 50.0 AVG: 6.509 Epoch: 664 -----------
--------- CURRENT: 50.0 AVG: 6.57 Epoch: 665 -----------
--------- CURRENT: 50.0 AVG: 6.63 Epoch: 666 -----------
--------- CURRENT: 50.0 AVG: 6.69 Epoch: 667 -----------
--------- CURRENT: 50.0 AVG: 6.75 Epoch: 668 -----------
--------- CURRENT: 27.5 AVG: 6.779 Epoch: 669 -----------
--------- CURREN

...

--------- CURRENT: -7.75 AVG: 3.735 Epoch: 7778 -----------
--------- CURRENT: 22.25 AVG: 3.737 Epoch: 7779 -----------
--------- CURRENT: 50.0 AVG: 3.743 Epoch: 7780 -----------
--------- CURRENT: 49.25 AVG: 3.749 Epoch: 7781 -----------
--------- CURRENT: 31.25 AVG: 3.752 Epoch: 7782 -----------
--------- CURRENT: 50.0 AVG: 3.757 Epoch: 7783 -----------
--------- CURRENT: -1.75 AVG: 3.757 Epoch: 7784 -----------
--------- CURRENT: 26.0 AVG: 3.759 Epoch: 7785 -----------
--------- CURRENT: 50.0 AVG: 3.765 Epoch: 7786 -----------
--------- CURRENT: -82.0 AVG: 3.755 Epoch: 7787 -----------
--------- CURRENT: -244.5 AVG: 3.725 Epoch: 7787 -----------
--------- CURRENT: 39.75 AVG: 3.729 Epoch: 7788 -----------
--------- CURRENT: 50.0 AVG: 3.735 Epoch: 7789 -----------
--------- CURRENT: -236.8 AVG: 3.706 Epoch: 7789 -----------
--------- CURRENT: -37.75 AVG: 3.701 Epoch: 7790 -----------
--------- CURRENT: -8.5 AVG: 3.699 Epoch: 7791 -----------
--------- CURRENT: 50.0 AVG: 3.705 Epoch: 7

--------- CURRENT: 50.0 AVG: 3.635 Epoch: 7911 -----------
--------- CURRENT: 32.0 AVG: 3.639 Epoch: 7912 -----------
--------- CURRENT: 37.25 AVG: 3.643 Epoch: 7913 -----------
--------- CURRENT: 50.0 AVG: 3.648 Epoch: 7914 -----------
--------- CURRENT: -0.25 AVG: 3.648 Epoch: 7915 -----------
--------- CURRENT: 41.0 AVG: 3.652 Epoch: 7916 -----------
--------- CURRENT: 32.0 AVG: 3.656 Epoch: 7917 -----------
--------- CURRENT: 21.5 AVG: 3.658 Epoch: 7918 -----------
--------- CURRENT: 42.5 AVG: 3.662 Epoch: 7919 -----------
--------- CURRENT: 50.0 AVG: 3.668 Epoch: 7920 -----------
--------- CURRENT: -234.2 AVG: 3.64 Epoch: 7920 -----------
--------- CURRENT: -6.5 AVG: 3.638 Epoch: 7921 -----------
--------- CURRENT: 50.0 AVG: 3.644 Epoch: 7922 -----------
--------- CURRENT: 29.0 AVG: 3.647 Epoch: 7923 -----------
--------- CURRENT: -77.5 AVG: 3.637 Epoch: 7924 -----------
--------- CURRENT: 49.25 AVG: 3.643 Epoch: 7925 -----------
--------- CURRENT: -48.25 AVG: 3.637 Epoch: 7926 --

--------- CURRENT: -10.0 AVG: 3.632 Epoch: 8044 -----------
--------- CURRENT: -22.0 AVG: 3.629 Epoch: 8045 -----------
--------- CURRENT: 20.75 AVG: 3.631 Epoch: 8046 -----------
--------- CURRENT: 29.0 AVG: 3.634 Epoch: 8047 -----------
--------- CURRENT: 7.25 AVG: 3.634 Epoch: 8048 -----------
--------- CURRENT: -40.75 AVG: 3.629 Epoch: 8049 -----------
--------- CURRENT: 50.0 AVG: 3.634 Epoch: 8050 -----------
--------- CURRENT: -2.5 AVG: 3.634 Epoch: 8051 -----------
--------- CURRENT: -89.5 AVG: 3.623 Epoch: 8052 -----------
--------- CURRENT: 31.25 AVG: 3.626 Epoch: 8053 -----------
--------- CURRENT: 50.0 AVG: 3.631 Epoch: 8054 -----------
--------- CURRENT: -74.5 AVG: 3.622 Epoch: 8055 -----------
--------- CURRENT: 50.0 AVG: 3.628 Epoch: 8056 -----------
--------- CURRENT: 44.75 AVG: 3.633 Epoch: 8057 -----------
--------- CURRENT: -225.3 AVG: 3.606 Epoch: 8057 -----------
--------- CURRENT: 25.0 AVG: 3.608 Epoch: 8058 -----------
--------- CURRENT: 29.0 AVG: 3.611 Epoch: 805

Excessive output truncated after 524307 bytes.

-28.0 AVG: 3.382 Epoch: 8281 -----------

In [21]:
#θ = convert(Matrix, CSV.read("current473.csv"))

In [1]:
xx = []
yy = []
yyy = []
for (i, t) in enumerate(totals)
    push!(xx, i)
    push!(yy, t)
    push!(yyy, mean(yy))
end

plot(xx, yyy, ylims = (-500,50))

UndefVarError: UndefVarError: totals not defined

In [11]:
#thetaframe = CSV.read("0.44_theta_12.1_1.csv");


In [12]:
#θ = convert(Matrix, thetaframe)

In [25]:
thetaframe = DataFrame(θ)
CSV.write("0.44_theta_11.28_1", thetaframe)

"0.44_theta_11.28_1"

In [51]:
b = ParticleCollection([[20, 60, 90, 1] for i in 1:N]);
ξ = weighted_grid_2(b)/N
var(ξ)


3.5612535612535624e-5